In [1]:
# Model will produce a dataframe and csv file
# 1) at least season average 7tgts per game AND 
# 2) at least 7tgs over last three games AND 
# 3) season average at least 10yds per reception 

In [2]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os

In [3]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [4]:
# Function to get the current NFL week and year
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Reset this date at the start of the NFL season
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Set the current NFL year and week
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [5]:
# Base columns for all positions
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name',
    'position', 'position_group', 'recent_team', 'opponent_team',
    'fantasy_points', 'fantasy_points_ppr'
]

# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]


# Define the WR-specific target-related columns
wr_target_columns = ['targets', 'target_share', 'receptions', 'receiving_yards', 'receiving_tds']

# Combine base columns with WR-specific columns
wr_all_columns = base_columns + wr_columns
wr_tgt_columns = base_columns + wr_target_columns

# Import data for all the specified years
years = list(range(2017, current_year + 1))
nfl_data_all_years = nfl.import_weekly_data(
    years=years,
    columns=wr_all_columns
)

# Filter to show only WR positions
wr_data_all_years = nfl_data_all_years[nfl_data_all_years['position'] == 'WR']

# Select only the relevant columns for WRs
wr_tgt_data_all_years = wr_data_all_years[wr_tgt_columns]

Downcasting floats.


In [6]:
# TEST modifications worked
# Function returns WRs with at least 7 targets per game over the entire season with cumulative stats
def wr_tgts_7plus_per_game_current_season():
    # Step 1: Filter the data to include only the current season (current_year)
    wr_current_season = wr_tgt_data_all_years[wr_tgt_data_all_years['season'] == current_year]
    
    # Step 2: Get the most recent week (current NFL week)
    current_nfl_week = get_current_week()

    # Step 3: Group by player to sum cumulative stats (targets, receptions, receiving yards, receiving_tds) and calculate games played
    wr_grouped = wr_current_season.groupby(['season', 'player_id', 'player_name', 'recent_team'], as_index=False).agg({
        'targets': 'sum',               # Total targets over the season
        'receptions': 'sum',            # Total receptions over the season
        'receiving_yards': 'sum',       # Total receiving yards over the season
        'receiving_tds': 'sum',         # Total receiving touchdowns over the season
        'target_share': 'mean',         # Average target share over the season
        'week': 'count'                 # Number of games played (count of weeks)
    }).rename(columns={'week': 'games_played', 'receiving_tds': 'total_receiving_tds'})  # Rename receiving_tds here

    # Step 4: Calculate targets per game, receptions per game, receiving yards per game
    wr_grouped['targets_per_game'] = wr_grouped['targets'] / wr_grouped['games_played']
    wr_grouped['receptions_per_game'] = wr_grouped['receptions'] / wr_grouped['games_played']
    wr_grouped['receiving_yards_per_game'] = wr_grouped['receiving_yards'] / wr_grouped['games_played']

    # Step 5: Filter to keep only WRs with an average of 7 or more targets per game
    wr_filtered = wr_grouped[wr_grouped['targets_per_game'] >= 7].copy()  # Use .copy() to avoid the warning

    # Step 6: Rename 'player_id' to 'nflpy_player_id'
    wr_filtered = wr_filtered.rename(columns={'player_id': 'nflpy_player_id'})

    # Step 7: Add the 'week' column with the current NFL week for all rows using .loc[] to avoid SettingWithCopyWarning
    wr_filtered.loc[:, 'week'] = current_nfl_week

    # Step 8: Round the values of the specified columns
    wr_filtered['targets_per_game'] = wr_filtered['targets_per_game'].round(1)
    wr_filtered['receptions_per_game'] = wr_filtered['receptions_per_game'].round(1)
    wr_filtered['receiving_yards_per_game'] = wr_filtered['receiving_yards_per_game'].round(1)
    wr_filtered['target_share'] = wr_filtered['target_share'].round(3)
    wr_filtered['receiving_yards'] = wr_filtered['receiving_yards'].astype(int)  # Ensure no decimals for receiving_yards

    # Step 9: Reorder the columns and include 'total_receiving_tds'
    wr_filtered = wr_filtered[['season', 'week', 'nflpy_player_id', 'player_name', 'games_played', 'recent_team',
                               'targets', 'targets_per_game', 'target_share', 'receptions', 'receptions_per_game', 
                               'receiving_yards', 'receiving_yards_per_game', 'total_receiving_tds']]  # Already renamed

    # Step 10: Sort by targets per game in descending order
    wr_sorted = wr_filtered.sort_values(by='targets_per_game', ascending=False)

    # Step 11: Show only the first 5 rows
    return wr_sorted.head()


In [7]:
# TEST modifications

In [8]:
# Filter to show only WR positions
wr_data_all_years = nfl_data_all_years[nfl_data_all_years['position'] == 'WR']

# Select only the relevant columns for WRs
wr_tgt_data_all_years = wr_data_all_years[wr_tgt_columns]

# Call the function to get the filtered WRs for the current season 
wr_filtered_season = wr_tgts_7plus_per_game_current_season()

# Display the filtered data for verification
display(HTML(wr_filtered_season.head().to_html(index=False)))

season,week,nflpy_player_id,player_name,games_played,recent_team,targets,targets_per_game,target_share,receptions,receptions_per_game,receiving_yards,receiving_yards_per_game,total_receiving_tds
2024,8,00-0039337,M.Nabers,5,NYG,60,12.0,0.381,39,7.8,427,85.4,3
2024,8,00-0033908,C.Kupp,3,LA,35,11.7,0.298,23,7.7,198,66.0,2
2024,8,00-0037740,G.Wilson,7,NYJ,75,10.7,0.286,46,6.6,460,65.7,3
2024,8,00-0039067,R.Rice,3,KC,29,9.7,0.328,24,8.0,288,96.0,2
2024,8,00-0038117,W.Robinson,7,NYG,67,9.6,0.286,43,6.1,303,43.3,2


In [9]:
# Update the baseline dataframe by adding boolean columns for Criteria #2 and #3
def update_baseline_with_criteria_flags():
    # Baseline dataframe for the current season
    wr_current_season = wr_tgts_7plus_per_game_current_season()

    # Step 1: Filter the data for the current season and only include rows where players have targets (active games)
    wr_active_games = wr_tgt_data_all_years[(wr_tgt_data_all_years['season'] == current_year) & (wr_tgt_data_all_years['targets'] > 0)]

    # Step 2: Sort the data by player, then by week in descending order to get the most recent games first
    wr_active_games = wr_active_games.sort_values(by=['player_name', 'week'], ascending=[True, False])

    # Step 3: For each player, select the last 3 games where they had stats
    wr_last_3_games = wr_active_games.groupby('player_name').head(3)

    # Step 4: Group by player and sum the targets over the last 3 games
    wr_last_3_games_grouped = wr_last_3_games.groupby(['player_name', 'player_id'], as_index=False).agg({
        'targets': 'sum'  # Total targets over the last 3 games
    }).rename(columns={'targets': 'total_targets_last_3_games'})

    # Step 5: Merge the total targets for the last 3 games back into the current season dataframe
    wr_current_season = wr_current_season.merge(wr_last_3_games_grouped, left_on=['player_name', 'nflpy_player_id'], 
                                                right_on=['player_name', 'player_id'], how='left')

    # Drop 'player_id' from the right-hand dataframe after merge to avoid confusion
    wr_current_season.drop(columns=['player_id'], inplace=True)

    # Step 6: Calculate 'avg_7tgs_last3_games' as total targets over last 3 games divided by 3
    wr_current_season['avg_7tgs_last3_games'] = (wr_current_season['total_targets_last_3_games'] / 3).round(2)

    # Step 7: Create the 'avg_7tgs_last3_games_bool' column (TRUE if average targets per game over the last 3 games >= 7)
    wr_current_season['avg_7tgs_last3_games_bool'] = wr_current_season['avg_7tgs_last3_games'] >= 7
    wr_current_season['avg_7tgs_last3_games_bool'] = wr_current_season['avg_7tgs_last3_games_bool'].fillna(False)

    # Step 8: Add 'season_avg_10yds_per_reception_bool' (TRUE if season average yards per reception >= 10)
    wr_current_season['season_avg_10yds_per_reception'] = (wr_current_season['receiving_yards'] / wr_current_season['receptions']).round(2)
    wr_current_season['season_avg_10yds_per_reception_bool'] = wr_current_season['season_avg_10yds_per_reception'] >= 10

    # Only drop 'meets_criteria_3' if it exists in the dataframe
    if 'meets_criteria_3' in wr_current_season.columns:
        wr_current_season.drop(columns=['meets_criteria_3'], inplace=True)

    # Display the first 5 rows to verify the boolean columns
    display(wr_current_season.head())

    # Return the updated baseline dataframe with the criteria flags
    return wr_current_season

# Call the function to verify the output
wr_with_criteria_flags = update_baseline_with_criteria_flags()


,season,week,nflpy_player_id,player_name,games_played,recent_team,targets,targets_per_game,target_share,receptions,receptions_per_game,receiving_yards,receiving_yards_per_game,total_receiving_tds,total_targets_last_3_games,avg_7tgs_last3_games,avg_7tgs_last3_games_bool,season_avg_10yds_per_reception,season_avg_10yds_per_reception_bool
0,2024,8,00-0039337,M.Nabers,5,NYG,60,12.0,0.381,39,7.8,427,85.4,3,35,11.67,True,10.95,True
1,2024,8,00-0033908,C.Kupp,3,LA,35,11.7,0.298,23,7.7,198,66.0,2,35,11.67,True,8.61,False
2,2024,8,00-0037740,G.Wilson,7,NYJ,75,10.7,0.286,46,6.6,460,65.7,3,41,13.67,True,10.00,True
3,2024,8,00-0039067,R.Rice,3,KC,29,9.7,0.328,24,8.0,288,96.0,2,29,9.67,True,12.00,True
4,2024,8,00-0038117,W.Robinson,7,NYG,67,9.6,0.286,43,6.1,303,43.3,2,29,9.67,True,7.05,False


In [10]:
# test


In [11]:
# test


In [12]:
# test


In [13]:
# test
